# Linear Programming and MILP Optimization With Python

This notebook demonstrates the use of Mixed Integer Linear Programming (MILP) to solve this optimization problem. The goal is to allocate different rice varieties to different field types to maximize the total harvest yield. Pyomo and OR-tools are used together with the linear and PLEX solvers.

**1. Using Google OR-tools solver**

**a) Formulate and solve the optimization problem.**

In [1]:
# Set up OR-tools library
from ortools.linear_solver import pywraplp

In [2]:
# Create the MILP solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

In [3]:
# Define decision variables
fields = ['I', 'II', 'III']
rice_types = ['IR8', 'CBC', 'IR132']
x = {}
for f in fields:
    for r in rice_types:
        x[(f, r)] = solver.IntVar(0, solver.infinity(), f'x_{f}_{r}')


In [4]:
# Objective function
yield_matrix = {'I': {'IR8': 4, 'CBC': 6, 'IR132': 5},
                'II': {'IR8': 8, 'CBC': 9, 'IR132': 4},
                'III': {'IR8': 6, 'CBC': 7, 'IR132': 6}}

solver.Maximize(solver.Sum(x[(f, r)] * yield_matrix[f][r] for f in fields for r in rice_types))

# Constraints for total hectares for each rice type
solver.Add(solver.Sum(x[(f, 'IR8')] for f in fields) == 30)
solver.Add(solver.Sum(x[(f, 'CBC')] for f in fields) == 20)
solver.Add(solver.Sum(x[(f, 'IR132')] for f in fields) == 40)

# Constraints for total hectares for each field type
solver.Add(solver.Sum(x[('I', r)] for r in rice_types) == 25)
solver.Add(solver.Sum(x[('II', r)] for r in rice_types) == 25)
solver.Add(solver.Sum(x[('III', r)] for r in rice_types) == 40)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x0000026FCE275F50> >

In [16]:
# Solve the problem
status = solver.Solve()

# Output results
if status == pywraplp.Solver.OPTIMAL:
    print("Status:", "Optimal")
    for f in fields:
        for r in rice_types:
            print(f"Field {f}, Rice {r}: {x[(f, r)].solution_value()} hectares")
else:
    print("The problem does not have an optimal solution.")

Status: Optimal
Field I, Rice IR8: 0.0 hectares
Field I, Rice CBC: 20.0 hectares
Field I, Rice IR132: 5.0 hectares
Field II, Rice IR8: 25.0 hectares
Field II, Rice CBC: 0.0 hectares
Field II, Rice IR132: 0.0 hectares
Field III, Rice IR8: 5.0 hectares
Field III, Rice CBC: 0.0 hectares
Field III, Rice IR132: 35.0 hectares


**b) Additional scenario where IR8 cannot be grown on Field Type I**

In [17]:
solver.Add(x[('I', 'IR8')] == 0)

# Solve the problem
status = solver.Solve()

# Output results
if status == pywraplp.Solver.OPTIMAL:
    print("Status:", "Optimal")
    for f in fields:
        for r in rice_types:
            print(f"Field {f}, Rice {r}: {x[(f, r)].solution_value()} hectares")
else:
    print("The problem does not have an optimal solution.")

Status: Optimal
Field I, Rice IR8: 0.0 hectares
Field I, Rice CBC: 20.0 hectares
Field I, Rice IR132: 5.0 hectares
Field II, Rice IR8: 25.0 hectares
Field II, Rice CBC: 0.0 hectares
Field II, Rice IR132: 0.0 hectares
Field III, Rice IR8: 5.0 hectares
Field III, Rice CBC: 0.0 hectares
Field III, Rice IR132: 35.0 hectares


**2. Using CPLEX solver**

**a) Formulate and solve the optimization problem.**

In [5]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, NonNegativeIntegers, SolverFactory, maximize

# Create a new model
model = ConcreteModel()

In [6]:
# Sets
fields = ['I', 'II', 'III']
rice_types = ['IR8', 'CBC', 'IR132']

# Decision variables
model.x = Var(fields, rice_types, domain=NonNegativeIntegers)

In [11]:
# Objective: Maximize yield
yield_coefficients = {
    ('I', 'IR8'): 4, ('I', 'CBC'): 6, ('I', 'IR132'): 5,
    ('II', 'IR8'): 8, ('II', 'CBC'): 9, ('II', 'IR132'): 4,
    ('III', 'IR8'): 6, ('III', 'CBC'): 7, ('III', 'IR132'): 6
}
model.objective = Objective(expr=sum(model.x[f, r] * yield_coefficients[(f, r)] for f in fields for r in rice_types), sense=maximize)

# Constraints
# Total hectares for each rice type
model.total_hectares_ir8 = Constraint(expr=sum(model.x[f, 'IR8'] for f in fields) == 30)
model.total_hectares_cbc = Constraint(expr=sum(model.x[f, 'CBC'] for f in fields) == 20)
model.total_hectares_ir132 = Constraint(expr=sum(model.x[f, 'IR132'] for f in fields) == 40)

# Total hectares for each field type
model.total_field_i = Constraint(expr=sum(model.x['I', r] for r in rice_types) == 25)
model.total_field_ii = Constraint(expr=sum(model.x['II', r] for r in rice_types) == 25)
model.total_field_iii = Constraint(expr=sum(model.x['III', r] for r in rice_types) == 40)

'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
(type=<class 'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
(type=<class 'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
(type=<class 'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.Abs

In [9]:
# Solve the model using CPLEX
solver = SolverFactory('cplex', executable ='C:/Program Files/IBM/ILOG/CPLEX_Studio_Community2211/cplex/bin/x64_win64/cplex.exe')
result = solver.solve(model, tee=True)

# Print the results
if result.solver.status == 'ok' and result.solver.termination_condition == 'optimal':
    print("Solution status:", result.solver.status)
    for f in fields:
        for r in rice_types:
            print(f"Field {f}, Rice {r}: {model.x[f, r].value} hectares")
elif result.solver.termination_condition == 'infeasible':
    print("No feasible solution found.")
else:
    print("Solver Status: ", result.solver.status)

executable for solver cplex. File with name=C:/Program
Files/IBM/ILOG/CPLEX_Studio_Community2211/cplex/bin/x64_win64/cplex.exe either
does not exist or it is not executable. To skip this validation, call
set_executable with validate=False.
Traceback (most recent call last):
  File "c:\Users\Orioki\Desktop\Hackathon\Linear-Programming-MILP-DP-NLP-Optimization-With-Python\env\Lib\site-packages\pyomo\opt\base\solvers.py", line 148, in __call__
    opt = self._cls[_name](**kwds)
          ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Orioki\Desktop\Hackathon\Linear-Programming-MILP-DP-NLP-Optimization-With-Python\env\Lib\site-packages\pyomo\solvers\plugins\solvers\CPLEX.py", line 164, in __init__
    ILMLicensedSystemCallSolver.__init__(self, **kwds)
  File "c:\Users\Orioki\Desktop\Hackathon\Linear-Programming-MILP-DP-NLP-Optimization-With-Python\env\Lib\site-packages\pyomo\opt\solver\ilmcmd.py", line 29, in __init__
    pyomo.opt.solver.shellcmd.SystemCallSolver.__init__(self, **kwds)
  File 

RuntimeError: Attempting to use an unavailable solver.

The SolverFactory was unable to create the solver "_cplex_shell"
and returned an UnknownSolver object.  This error is raised at the point
where the UnknownSolver object was used as if it were valid (by calling
method "solve").

The original solver was created with the following parameters:
	executable: C:/Program Files/IBM/ILOG/CPLEX_Studio_Community2211/cplex/bin/x64_win64/cplex.exe
	type: _cplex_shell
	_args: ()
	options: {}

**b) Additional scenario where IR8 cannot be grown on Field Type I**

In [11]:
# Additional scenario where IR8 cannot be grown on Field Type I
model.no_ir8_field_i = Constraint(expr=model.x['I', 'IR8'] == 0)

# Solve the model using CPLEX
solver = SolverFactory('cplex', executable ='C:/Program Files/IBM/ILOG/CPLEX_Studio_Community2211/cplex/bin/x64_win64/cplex.exe')
result = solver.solve(model, tee=True)

# Print the results
if result.solver.status == 'ok' and result.solver.termination_condition == 'optimal':
    print("Solution status:", result.solver.status)
    for f in fields:
        for r in rice_types:
            print(f"Field {f}, Rice {r}: {model.x[f, r].value} hectares")  # Changed from .value() to .value
elif result.solver.termination_condition == 'infeasible':
    print("No feasible solution found.")
else:
    print("Solver Status: ", result.solver.status)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer Community Edition 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\huyha\AppData\Local\Temp\tmpt62jq_du.cplex.log' open.
CPLEX> Problem 'C:\Users\huyha\AppData\Local\Temp\tmplvvdyrdt.pyomo.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> Problem name         : C:\Users\huyha\AppData\Local\Temp\tmplvvdyrdt.pyomo.lp
Objective sense      : Maximize
Variables            :       9  [General Integer: 9]
Objective nonzeros   :       9
Linear constraints   :       7  [Equal: 7]
  Nonzeros           :      19
  RHS nonzeros       :       6

Variables            : Min LB: 0.000000         Max UB: all infinite   
Objective nonzeros   : Min 

**3. Original Problem #8 from tstran155**

**Using pulp to solve**

In [13]:
# a) Solve the problem as described

import pulp

# Create the LP problem
prob = pulp.LpProblem("LP_Problem_8a", pulp.LpMaximize)

# Define the decision variables
rice_type = [30, 20, 40]
field_type = [25, 25, 40]
productivity8a = [[4, 8, 6], [6, 9, 7], [5, 4, 6]] #tons/hectare

dim_i = len(rice_type)
dim_j = len(field_type)

# Productivity per hectares rice field
x = [[pulp.LpVariable("x_{}_{}".format(i, j), lowBound=0, cat="Integer") for j in range(dim_j)] for i in range(dim_i)]

# Define the objective function
prob += pulp.lpSum([[productivity8a[i][j] * x[i][j] for i in range(dim_i)] for j in range(dim_j)])

# Define the constraints
for i in range(dim_i):
    prob += pulp.lpSum([x[i][j] for j in range(dim_j)]) == rice_type[i]
    
for j in range(dim_j):
    prob += pulp.lpSum([x[i][j] for i in range(dim_i)]) == field_type[j]

# Solve the LP problem
status = prob.solve()

# Print the solution
print(f"Status: {pulp.LpStatus[status]}")
print(f"Optimal value: {pulp.value(prob.objective)}")
print("Optimal solution: ")
for i in range(dim_i):
    for j in range(dim_j):
        print(f"x[{i}][{j}] = {x[i][j].varValue}")

Status: Optimal
Optimal value: 585.0
Optimal solution: 
x[0][0] = 0.0
x[0][1] = 25.0
x[0][2] = 5.0
x[1][0] = 20.0
x[1][1] = 0.0
x[1][2] = 0.0
x[2][0] = 5.0
x[2][1] = 0.0
x[2][2] = 35.0


In [14]:
# b) Formulate and solve the problem in the case where IR8 rice cannot be grown on type I field

import pulp

# Create the LP problem
prob = pulp.LpProblem("LP_Problem_8b", pulp.LpMaximize)

# Define the decision variables
rice_type = [30, 20, 40]
field_type = [25, 25, 40]
productivity8b = [[0, 8, 6], [6, 9, 7], [5, 4, 6]] #tons/hectare

dim_i = len(rice_type)
dim_j = len(field_type)

# Productivity per hectares rice field
x = [[pulp.LpVariable("x_{}_{}".format(i, j), lowBound=0, cat="Integer") for j in range(dim_j)] for i in range(dim_i)]

# Define the objective function
prob += pulp.lpSum([[productivity8b[i][j] * x[i][j] for i in range(dim_i)] for j in range(dim_j)])


# Define the constraints
for i in range(dim_i):
    prob += pulp.lpSum([x[i][j] for j in range(dim_j)]) == rice_type[i]
    
for j in range(dim_j):
    prob += pulp.lpSum([x[i][j] for i in range(dim_i)]) == field_type[j]

# Solve the LP problem
status = prob.solve()

# Print the solution
print(f"Status: {pulp.LpStatus[status]}")
print(f"Optimal value: {pulp.value(prob.objective)}")
print("Optimal solution: ")
for i in range(dim_i):
    for j in range(dim_j):
        print(f"x[{i}][{j}] = {x[i][j].varValue}")

Status: Optimal
Optimal value: 585.0
Optimal solution: 
x[0][0] = 0.0
x[0][1] = 25.0
x[0][2] = 5.0
x[1][0] = 20.0
x[1][1] = 0.0
x[1][2] = 0.0
x[2][0] = 5.0
x[2][1] = 0.0
x[2][2] = 35.0


**3. Discussions**

a. OR-tools solver:

Results of the first and second scenario where IR8 cannot be grown on Field Type I are indeed identical. The unchanged solution when adding the constraint that IR8 cannot be grown in Field I suggests two things: firstly, that the restriction aligns with the optimal planting strategy derived without it, and secondly, it could indicate a potential area of focus for further investigation like planting strategy, costs, or yields could change this result.

The solutions provided by the LP and MILP techniques are different. Here is the insights:

Allocation of Rice Types:

- IR8: The LP solution does not utilize IR8 in Field II and uses all 25 hectares in Field III. In contrast, the MILP solution fully utilizes Field II for IR8 planting, which suggests different optimization dynamics between integer constraints (MILP) and continuous variables (LP).
    
- IR132: The LP solution has a significant allocation of IR132 in Field II (20 hectares), which is not present in the MILP solution. This again reflects the flexibility in continuous allocation in LP that might not be as effective or feasible when restricted to integer values in MILP.

Field Utilization:

- Field II in LP has only IR132 planted and utilizes only 20 out of 25 possible hectares, indicating a potentially suboptimal use of available land under the constraints and yield assumptions of LP.
    
- Field III shows differing allocations between IR8 and IR132 across the solutions, which might indicate that the profitability or yield per hectare changes dramatically based on how the constraints are treated (integer vs. continuous).

b. CPLEX solver:

- The optimal value reached is 585.0, which represents the maximum total yield based on your model's constraints and objective function.
  
- Rice IR132 is heavily allocated to Field I, which might indicate it's the most profitable or effective use of that field, while rice CBC is primarily allocated to Field II, with Rice IR8 also placed significantly in Field III and minimally in Field II.
  
- The results are reasonable and align with typical outcomes in agricultural optimization, where certain crops are prioritized in specific fields to optimize the use of available resources and meet demand constraints effectively.
  
c. LP vs. MILP:

- LP offers more flexibility in how variables can be allocated, potentially allowing for more "fine-tuned" solutions at a fractional level, which might not always be practical or feasible in real-world scenarios.
  
- MILP provides solutions that are more practical for real-world applications where the decision variables must be whole numbers (like the number of hectares that can realistically be planted).
  
- Depending on the exact nature of the constraints and objective function, MILP might yield a less optimal solution in terms of the raw numbers but a more feasible and practically implementable solution.
